### Compare Data Query Methods

Compare query, save, and load times of expression matrices between csv, ga4gh, xena, python pickle, hdf5

http://matthewrocklin.com/blog/work/2015/03/16/Fast-Serialization

http://stackoverflow.com/questions/37010212/what-is-the-fastest-way-to-upload-a-big-csv-file-in-notebook-to-work-with-python/37012035#37012035

http://people.duke.edu/~ccc14/sta-663-2016/A09_Intermediate_Sized_Data.html

In [1]:
# Primarily we're concerned with getting what we want into a pandas dataframe
import numpy as np
import pandas as pd

In [82]:
%%time
# Start with straight tsv being careful to load directly as float32 vs. pandas float64 default
from collections import defaultdict
converters = defaultdict(str)
converters["Gene"] = str
expression = pd.read_csv("treehouse/expression.tsv.gz", sep="\t", index_col=0, converters=converters, dtype=np.float32)
print "Type: {} Total Size Bytes: {}".format(expression.ix[1,1], expression.values.nbytes)
print "Shape: {}".format(expression.shape)

Type: 5.72339487076 Total Size Bytes: 1175483880
Shape: (27165, 10818)
CPU times: user 2min 8s, sys: 1.34 s, total: 2min 10s
Wall time: 2min 10s


In [83]:
%%time
# Save as pickle via pandas
expression.to_pickle("treehouse/expression.pkl")
!ls -alh data/expression.pkl

ls: cannot access data/expression.pkl: No such file or directory
CPU times: user 348 ms, sys: 1.64 s, total: 1.98 s
Wall time: 11.4 s


In [2]:
%%time
# Read back pickle using pandas
expression = pd.read_pickle("treehouse/expression.pkl")

CPU times: user 320 ms, sys: 904 ms, total: 1.22 s
Wall time: 1.22 s


In [3]:
# Needed to store to hdf5
!pip2 install --quiet tables

In [86]:
%%time
# Save as hdf5 via pandas
expression.to_hdf("treehouse/expression.hd5", "expression", mode="w", format="fixed")
!ls -alh data/expression.hd5

ls: cannot access data/expression.hd5: No such file or directory
CPU times: user 2.85 s, sys: 1.78 s, total: 4.62 s
Wall time: 4.76 s


In [8]:
%%time
# Read back hdf using pandas
expression = pd.read_hdf("treehouse/expression.hd5", "expression")

CPU times: user 20 ms, sys: 600 ms, total: 620 ms
Wall time: 619 ms


In [5]:
expression.shape

(27165, 10818)

In [42]:
# from sqlalchemy import create_engine
# con = create_engine('sqlite:///:memory:')

import sqlite3
con = sqlite3.connect(':memory:')

# Write to sqlite database - NOTE: Only doing 900 genes more then this causes a SQL error...
from pandas.io import sql
expression.T.ix[0:10,0:900].to_sql(con=con, name='expression', if_exists='append')

for row in con.execute("SELECT * FROM expression LIMIT 1"):
        print row

(u'icgc/_EGAR00001415737_RNA_PAIRED_ICGC_GBM15_tumor_SN935_0182_B_C2UKHACXXs_131105_1', 8.510573387145996, 8.179963111877441, 5.3575520515441895, 14.5289945602417, 7.170544147491455, 7.00307035446167, 4.026313304901123, 4.1479010581970215, 7.324384689331055, 3.5739924907684326, 10.883962631225586, 2.769390821456909, 0.0, 2.423201560974121, 2.2130050659179688, 9.772239685058594, 9.483633041381836, 12.724106788635254, 9.365360260009766, 3.8166918754577637, 3.6595137119293213, 14.062152862548828, 11.607120513916016, 7.317823886871338, 9.870930671691895, 3.2820394039154053, 4.611172199249268, 7.565126419067383, 13.95543384552002, 6.933014392852783, 2.7032556533813477, 0.8304175734519958, 7.6017231941223145, 15.042986869812012, 7.109323024749756, 8.99187183380127, 7.257387638092041, 8.380952835083008, 8.224878311157227, 13.359524726867676, 13.143766403198242, 5.331730365753174, 11.855903625488281, 7.508514881134033, 9.109236717224121, 8.95075511932373, 7.135826110839844, 1.6698431968688965,